# Project Desription
#### PROBLEM STATEMENT <i>(according to the data provider on Kaggle)</i>     

"India is one of the fastest developing nations of the world and trade between nations is the major component of any developing nation. This dataset includes the trade data for India for commodities in the HS2 basket.The dataset consists of trade values for export and import of commodities in million US$."

##### GOAL OF THE PROJECT    

We are required to model the trade between India and other countries, and do Hypothesis Testing on various exploratory questions and make efficient visualizations to convey the results of those testing results.  

##### DATA SOURCE  
* FROM Kaggle - https://www.kaggle.com/lakshyaag/india-trade-data#2018-2010_import.csv

**Okay so prima facie, lets import some of the libraries we will be needing for this project.**  
(even if we are missing out on some library, we can always import it later on in the project.)

In [2]:
import sys

# library to handle vectorized data 
import numpy as np 

# library for data analsysis and man

import pandas as pd 
# so that the output is not trunacated by pandas when we actually want to see it 
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 100000)

# for visualisations
import seaborn as sns

%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt


# Loading the data.

**Okay so we have the two following datasets, representing the EXPORT and IMPORT trade done by India from 2010 to 2018 respectively -**  
* 2018-2010_export.csv - export data
* 2018-2010_import.csv - import data

In [3]:
export_data = pd.read_csv('2018-2010_export.csv')
import_data = pd.read_csv('2018-2010_import.csv')

print(export_data.shape, import_data.shape) # let's see the size f our datasets.

(137023, 5) (76124, 5)


In [4]:
export_data.head()

,HSCode,Commodity,value,country,year
0,2,MEAT AND EDIBLE MEAT OFFAL.,0.18,AFGHANISTAN TIS,2018
1,3,"FISH AND CRUSTACEANS, MOLLUSCS AND OTHER AQUAT...",0.00,AFGHANISTAN TIS,2018
2,4,DAIRY PRODUCE; BIRDS' EGGS; NATURAL HONEY; EDI...,12.48,AFGHANISTAN TIS,2018
3,6,LIVE TREES AND OTHER PLANTS; BULBS; ROOTS AND ...,0.00,AFGHANISTAN TIS,2018
4,7,EDIBLE VEGETABLES AND CERTAIN ROOTS AND TUBERS.,1.89,AFGHANISTAN TIS,2018


In [5]:
import_data.head()

,HSCode,Commodity,value,country,year
0,5,"PRODUCTS OF ANIMAL ORIGIN, NOT ELSEWHERE SPECI...",0.00,AFGHANISTAN TIS,2018
1,7,EDIBLE VEGETABLES AND CERTAIN ROOTS AND TUBERS.,12.38,AFGHANISTAN TIS,2018
2,8,EDIBLE FRUIT AND NUTS; PEEL OR CITRUS FRUIT OR...,268.60,AFGHANISTAN TIS,2018
3,9,"COFFEE, TEA, MATE AND SPICES.",35.48,AFGHANISTAN TIS,2018
4,11,PRODUCTS OF THE MILLING INDUSTRY; MALT; STARCH...,NaN,AFGHANISTAN TIS,2018
